In [1]:
!git clone https://github.com/zzeiidann/FNN_Beta.git

Cloning into 'FNN_Beta'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 8), reused 15 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 10.64 KiB | 5.32 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
from FNN_Beta.model import FNN
from FNN_Beta.dataset import CachedBERTDataset
from keras.optimizers import SGD, Adam

In [3]:
texts = [
    "Makanan di restoran ini enak banget, pelayanannya juga ramah.",      # 1
    "Saya kecewa, paket pesanan datang terlambat dan rusak.",             # 0
    "Film itu bikin terharu, akting para pemainnya luar biasa.",          # 1
    "Aplikasi sering error, bikin kesal waktu dipakai.",                  # 0
    "Pemandangan di pantai ini sangat indah dan menenangkan.",            # 1
    "Produk ini tidak sesuai deskripsi, kualitasnya buruk sekali.",       # 0
    "Saya puas dengan layanan customer service yang cepat tanggap.",      # 1
    "Suara musiknya pecah dan bikin telinga sakit.",                      # 0
    "Hotel bersih, nyaman, dan dekat dengan pusat kota.",                 # 1
    "Koneksi internet sering putus, bikin kerjaan terganggu.",            # 0
]

labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [11]:
dataset = CachedBERTDataset(
    texts=texts,
    labels=labels,
    bert_model="indolem/indobert-base-uncased", # <- Pilih BERT Model sesuai kasus / dataset yang digunakan
    max_length=128,
    cuda=True,
    testing_mode=False
)

Loading BERT model: indolem/indobert-base-uncased


In [5]:
dims, _ = dataset[0]
print('Dims Input for Encoder', dims.shape)

Dims Input for Encoder torch.Size([768])


In [6]:
model = FNN(
            dims = [768, 2022, 2023, 32],
            n_clusters = 3,
            batch_size = 12
        )

In [7]:
encoder_weight = model.pretrain_autoencoder(dataset, epochs=100,  optimizer = 'sgd')

Pretraining autoencoder...
Converted dataset to numpy array with shape: (10, 768)
Epoch 1/100


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.8403
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.8399
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.8396
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8392
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8389
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.8385
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8382
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.8379
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.8375
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8372
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.8369
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8366
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.8363
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.8361
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.8358
Epoch 16/100
1/1 ━━━━━━━━━━━━━━━━

In [8]:
model.initialize_model(
    ae_weights = '/content/pretrained_ae.weights.h5', # <- cek path yah
    gamma = 0.5, # <- ini bobot loss topik / cluster
    eta = 0.5, # <- ini bobot loss sentiment
    optimizer = SGD(learning_rate=0.001, momentum=0.9) # <- bisa ganti optimizer / lr tergantung kemauan
)

Pretrained AE weights are loaded successfully.


In [9]:
model.train(
    dataset = dataset,
    tol = 1e-4,
    update_interval = 70,
    maxiter = 1000,
)

Update interval 70
Computed class weights: {np.int64(0): np.float64(1.0), np.int64(1): np.float64(1.0)}
Save interval 4.166666666666667

Start Training...
Initializing cluster centers with k-means.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Class negative accuracy: 0.4
Class positive accuracy: 0.0
Iter 0 : Cluster Loss 0 , Sentiment Loss 0 , Acc_sentiment 0.2 ; loss= [0 0 0]


Class negative accuracy: 1.0
Class positive accuracy: 1.0
Iter 70 : Cluster Loss 0.06843 , Sentiment Loss 0.14589 , Acc_sentiment 1.0 ; loss= [0.21431 0.06843 0.14589]


delta_label  0.0 < tol  0.0001
Reached tolerance threshold. Stopping training.
saving model to: ./results/fnnjst/FNN_model_final.weights.h5


In [10]:
model.predict('Kamu jelek', bert_model='indolem/indobert-base-uncased')

[{'sentiment': 'positive', 'cluster': 0}]